In [ ]:
from utils.sample import Sample
from scipy.optimize import brentq
import pandas as pd
import numpy as np
from utils.biseccion import bisec
from utils.black_scholes import raiz_ratio

In [ ]:
opn = Sample(ratio=[0.4, 1.6], T=[0.2, 1.1], r=[0.02, 0.1], o=[0.01, 1])

opn.create('prueba', N=10**5)
x_test, y_test = opn.open('prueba')

In [ ]:
df = pd.DataFrame(x_test, columns=['c/k', 'ratio', 'r', 'T'])
df['o'] = y_test
df.head()

In [ ]:
# Tiro los casos en que no se puede aplicar el metodo de biseccion
# y calculo la volatilidad implícita
vol = []
i = 0
drops = []
for c, ratio, r, T in x_test:
    f = lambda x: raiz_ratio(c, ratio, r, x, T)
    # que se cumpla la precondicción
    if f(0.01) < 0:
        # máxima precision
        o = bisec(f, 0.01, 1, 2**-56)
        vol.append(o)
    else: 
        drops.append(i)
    i += 1
    
    if i % 10000 == 0:
        print('{}%'.format(100*i/len(x_test)))
    
    
dfb = df.drop(drops)
#volatilidad implicita
dfb['o_est'] = vol

In [ ]:
# error entre la volatilidad estimada e implícita
dfb['diff'] = (dfb['o'] - dfb['o_est']).apply(abs)
print('Error absoluto medio biseccion: ', dfb['diff'].mean())

In [ ]:
#Ordeno el dataframe segun el error entre la volatilidad estimada
# y la volatilidad implícita
orda = dfb.sort_values('diff', ignore_index=True, ascending= False)

In [ ]:
#aplico la función que busca la raiz sobre la volatilidad generada
#mediante el método de bisección
fs = []
auxi = orda[['c/k','ratio','r', 'o_est', 'T']]
for i in range(len(auxi)):
    fs.append(raiz_ratio(*auxi.iloc[i]))

orda['f(o_est)'] = fs


In [ ]:
orda.head(50)

Observar q en muchos casos la diferencia entre la volatilidad implícita y la volatilidad estimada es grande, eso es porque en ciertos casos la volatilidad tiene poco impacto, osea un "vega bajo", y por el problema de precisión la función que calcula la raíz retorna 0.

In [ ]:
import scipy.stats as si
#vega en funcion de ratio osea la función retorna vega/K.
def vega(ratio, r, o, T):
    
    d1 = (np.log(ratio) + (r + 0.5 * o ** 2) * T) / (o * np.sqrt(T))
    
    vega = ratio * si.norm.cdf(d1, 0.0, 1.0) * np.sqrt(T)
    
    return vega

In [ ]:
def my_vega(elem):
    return vega(elem['ratio'], elem['r'], elem['o'], elem['T'])

orda['vega/K'] = orda.apply(my_vega, axis=1)

In [ ]:
orda.head(50)

In [ ]:
orda.tail()